# Supervised approach

We start by using a supervised approach, that is to say, we will use specific defined behaviors.

In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import pickle as pkl
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

In [15]:
# Set up directories
root = Path('/Volumes/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/1_SOC/1_ProtocolControlsMales')
deepof_path = root / "DeepOF/supervised_annotation.pkl"
deepof_conditions_path = root / "DeepOF/conditions.csv"

# Upload conditions
target_values = pd.read_csv(deepof_conditions_path)

# Upload raw DeepOF data
with open(deepof_path, 'rb') as f:
    deepof_data = pkl.load(f)

In [16]:
# Compte abundance means for each behavior, individual, and time bin
def compute_means(data, number_of_bins=6):
    means = {}
    for id, behaviors in data.items():
        means[id] = {}
        
        # Divide rows of dataframe into number of bins
        numeric_index = pd.RangeIndex(len(behaviors))
        time_bins = pd.cut(numeric_index, bins=number_of_bins, labels=range(number_of_bins))
        behaviors = behaviors.copy()
        behaviors['time_bin'] = time_bins
        
        # Compute means for each behavior in the specified time bins
        for time_bin in range(number_of_bins):
            means[id][time_bin] = {}
            bin_df = behaviors[behaviors['time_bin'] == time_bin]
            for behavior in behaviors.columns[:-2]:  # exclude time_bin and maybe 'target' column
                means[id][time_bin][behavior] = np.nanmean(bin_df[behavior])
    
    # Convert means to DataFrame for easier handling
    # Columns: [id, time_bin, behavior1, behavior2, ...]
    means_df = pd.DataFrame()
    for id, time_bins in means.items():
        for time_bin, behaviors in time_bins.items():
            row = {'id': id, 'time_bin': time_bin}
            row.update(behaviors)
            means_df = pd.concat([means_df, pd.DataFrame([row])], ignore_index=True)
    means_df = means_df.fillna(0)
    
    return means_df

abundance_df = compute_means(deepof_data)

In [17]:
# Add a column indicating the cue acording to the target values
abundance_df['learning'] = abundance_df['id'].map(target_values.set_index('experiment_id')['learning'])

# Add a column indicating the group acording to the target values
abundance_df['group'] = abundance_df['id'].map(target_values.set_index('experiment_id')['group'])

In [30]:
# Check for missing values in the abundance_df DataFrame
abundance_df.isnull().sum()

id            0
time_bin      0
climbing      0
sniffing      0
huddle        0
lookaround    0
speed         0
learning      0
group         0
dtype: int64

In [13]:
# Download combined dataframe to csv
abundance_df.to_csv(root / '/Users/mcanela/Desktop/Behavior paper review/deepof_abundance_mean.csv', index=False)

## Biological questions

- Is the protocol working as expected?
- Which behavioral changes are there happening upon cue presentation?
- Are there differences between the direct and mediated response?
- Is it worth measuring more than one behavior? Does one behavior predict as well as an array of behaviors? Does it increase sensitivity to detect change (last minutes)

# Unsupervised approach

In [2]:
# Set up directories
root = Path('/Volumes/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/1_SOC/2_ProtocolSexAge')
moseq_path = root / "MoSeq/2024_05_23-11_06_49/results"
deepof_conditions_path = root / "DeepOF/conditions.csv"

# Upload conditions
target_values = pd.read_csv(deepof_conditions_path)

In [3]:
# Get all syllables from each file
syllable_dict = {}
all_syllables = set()

for file in moseq_path.glob('*.csv'):
    syllables = pd.read_csv(file, usecols=['syllable'])['syllable'].values
    syllable_dict[file.stem] = syllables
    all_syllables.update(syllables)

In [21]:
# Create a DataFrame to store abundance syllables as columns
rows = []

for file, syllables in syllable_dict.items():
    
    # Split syllables into 6 parts
    all_splits = np.array_split(syllables, 6)
    
    # Count the number of occurrences of each syllable in each part
    for i, split in enumerate(all_splits):
        counts = Counter(split)
        row = {'id': file, 'time_bin': i}
        for syll, count in counts.items():
            if syll not in all_syllables:
                continue
            row[syll] = count
        rows.append(row)
        
syllable_df = pd.DataFrame(rows)
syllable_df = syllable_df.fillna(0)

# Order the columns
syllable_df = syllable_df[sorted(syllable_df.columns[2:]) + ['id', 'time_bin']]

In [22]:
syllable_df

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,58,62,id,time_bin
0,188.0,251.0,308.0,294.0,80.0,20.0,141.0,10.0,70.0,76.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20231117_Marc_ERC SOC S1_Females_box bc_03_01_...,0
1,228.0,409.0,62.0,315.0,43.0,20.0,26.0,70.0,99.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20231117_Marc_ERC SOC S1_Females_box bc_03_01_...,1
2,234.0,317.0,115.0,393.0,91.0,12.0,63.0,28.0,43.0,58.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20231117_Marc_ERC SOC S1_Females_box bc_03_01_...,2
3,255.0,215.0,905.0,90.0,1.0,13.0,4.0,11.0,1.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20231117_Marc_ERC SOC S1_Females_box bc_03_01_...,3
4,331.0,188.0,147.0,446.0,49.0,0.0,100.0,45.0,74.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20231117_Marc_ERC SOC S1_Females_box bc_03_01_...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
931,59.0,399.0,773.0,35.0,60.0,8.0,39.0,17.0,2.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20240705_Marc_ERC SOC light_Females_box hi_01_...,1
932,131.0,613.0,192.0,82.0,125.0,47.0,47.0,18.0,26.0,52.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20240705_Marc_ERC SOC light_Females_box hi_01_...,2
933,37.0,138.0,1206.0,14.0,8.0,9.0,0.0,0.0,7.0,38.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20240705_Marc_ERC SOC light_Females_box hi_01_...,3
934,132.0,195.0,960.0,22.0,40.0,18.0,8.0,10.0,17.0,44.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20240705_Marc_ERC SOC light_Females_box hi_01_...,4


In [23]:
# Split "id" column by "DLC" and keep the first part
syllable_df['id'] = syllable_df['id'].str.split('DLC').str[0]

# Add a column indicating the cue acording to the target values
syllable_df['learning'] = syllable_df['id'].map(target_values.set_index('experiment_id')['learning'])

# Add a column indicating the group acording to the target values
syllable_df['group'] = syllable_df['id'].map(target_values.set_index('experiment_id')['group'])

In [24]:
# Check for missing values in the abundance_df DataFrame
syllable_df.isnull().sum()

0           0
1           0
2           0
3           0
4           0
5           0
6           0
7           0
8           0
9           0
10          0
11          0
12          0
13          0
14          0
15          0
16          0
17          0
18          0
19          0
20          0
21          0
22          0
23          0
24          0
58          0
62          0
id          0
time_bin    0
learning    0
group       0
dtype: int64

In [25]:
# Download combined dataframe to csv
syllable_df.to_csv(root / '/Users/mcanela/Desktop/Behavior paper review/moseq_abundance_sexage_mean.csv', index=False)